In [2]:
# Based on  tutorial https://github.com/h2oai/h2o-tutorials/blob/master/h2o-world-2017/automl/Python/automl_regression_powerplant_output.ipynb
# Try to rerun model 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [17]:
# Needed to install Java JRE https://www.java.com/en/download/win10.jsp 
import h2o 
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.221-b11, mixed mode)


C:\Users\dmmma\Anaconda3\lib\site-packages\h2o\backend\server.py:369: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Starting server from C:\Users\dmmma\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\dmmma\AppData\Local\Temp\tmpn56p23nd
  JVM stdout: C:\Users\dmmma\AppData\Local\Temp\tmpn56p23nd\h2o_dmmma_started_from_python.out
  JVM stderr: C:\Users\dmmma\AppData\Local\Temp\tmpn56p23nd\h2o_dmmma_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_dmmma_c6eoe9
H2O cluster total nodes:,1
H2O cluster free memory:,247.5 Mb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


### Load data

In [6]:
# Setting 'parse_dates' in this case parses both dates and times
# These files are too large to commit so they're uploaded locally under `/exploring_models` but not pushed
train_raw = pd.read_csv('weather1_education_train.csv', parse_dates = ['timestamp'])
val_raw = pd.read_csv('weather1_education_test.csv', parse_dates = ['timestamp'])

In [15]:
train_data = train_raw.drop([ 'building_name', 'primary_space_usage', 'hour', 'year', 'month','weekday', 'date', 'timestamp', 'Unnamed: 0'], axis=1)
val_data = val_raw.drop([ 'building_name', 'primary_space_usage', 'hour', 'year', 'month','weekday', 'date', 'timestamp', 'Unnamed: 0'], axis=1)

train_data.head()


,area,electricity,TemperatureC,month_1,month_2,month_3,month_4,month_5,month_6,month_7,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,2777.0,5.0,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,2777.0,5.2,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,2777.0,5.2,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,2777.0,5.5,6.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,2777.0,6.0,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [16]:
val_data.head()

,area,electricity,TemperatureC,month_1,month_2,month_3,month_4,month_5,month_6,month_7,...,hour_23,wkday_0,wkday_1,wkday_2,wkday_3,wkday_4,wkday_5,wkday_6,PSU_PrimClass,PSU_UnivClass
0,6149.0,34.900002,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,6149.0,32.700000,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,6149.0,32.200000,5.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,6149.0,32.300000,6.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,6149.0,32.599998,7.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


### Fit a model

In [18]:
# Using one train/val split, not walk-forward cross-val
# In h2o, if using all other cols for x, don't have to explicitly specify
# Takes > 1 min to run

# Must be formatted as h2o frames http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/frame.html
train =  h2o.H2OFrame(train_data)
test = h2o.H2OFrame(val_data)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [26]:
train.columns[:10]

['area',
 'electricity',
 'TemperatureC',
 'month_1',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7']

In [27]:
# In past, running for 60 s only fit one model (GLM), 3 min fit 3 (GLM, GBM, and Stacked Ensemble)
y = 'electricity'
x = list(train.columns)
x.remove(y)


In [32]:
aml =  H2OAutoML(max_runtime_secs = 60*5, seed = 1)
aml.train(x = x, y = y, training_frame = train) 
# https://h2o-release.s3.amazonaws.com/h2o/rel-xia/1/docs-website/h2o-docs/automl.html
# "If the user doesn’t specify a validation_frame, then one will be created automatically by randomly partitioning the training data"

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [33]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GLM_grid_1_AutoML_20190815_155034_model_1,901.617,30.0269,901.617,18.622,1.26586


In [36]:
# Inspect the leader model
aml.leader

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_grid_1_AutoML_20190815_155034_model_1

GLM Model: summary



,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,gaussian,identity,Ridge ( lambda = 74.835 ),"nlambda = 30, lambda.max = 2100.8, lambda.min = 74.835, lambda.1se = 74.835",78,78,9,automl_training_Key_Frame__upload_91d9561e91273f3ca426d42c2cb640b8.hex




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 898.5796696298365
RMSE: 29.976318480257653
MAE: 18.57536986807332
RMSLE: 1.2636314517335303
R^2: 0.016932709866141593
Mean Residual Deviance: 898.5796696298365
Null degrees of freedom: 437849
Residual degrees of freedom: 437771
Null deviance: 400219916.0688677
Residual deviance: 393443108.3474239
AIC: 4220461.484107741

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 901.6173790901555
RMSE: 30.02694421832091
MAE: 18.622017728736793
RMSLE: 1.2658643864683035
R^2: 0.01360938427988967
Mean Residual Deviance: 901.6173790901555
Null degrees of freedom: 437849
Residual degrees of freedom: 437771
Null deviance: 400219920.4307822
Residual deviance: 394773169.4346246
AIC: 4221939.169398433
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,18.6889,0.0681193,18.503267,18.690994,18.734331,18.770947,18.744963
mean_residual_deviance,905.9958,5.1672626,892.3205,904.82526,909.91766,912.79443,910.1213
mse,905.9958,5.1672626,892.3205,904.82526,909.91766,912.79443,910.1213
null_deviance,80043984.0000000,89050.21,79874016.0000000,79916616.0000000,80106560.0000000,8.01982E7,80124528.0000000
r2,0.0088280,0.0047032,0.0217030,0.0085221,0.0053063,0.0033017,0.0053069
residual_deviance,79338056.0000000,452497.2,78140504.0000000,79235544.0000000,79681488.0000000,79933408.0000000,79699320.0000000
rmse,30.099518,0.0860251,29.871735,30.080313,30.16484,30.212488,30.168217
rmsle,1.2690796,0.0032268,1.2602208,1.2693118,1.2713462,1.2725986,1.2719206


Scoring History: 


,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se
,2019-08-15 15:51:10,0.000 sec,1,.21E4,76,913.4931216,nan,913.6058648,0.7189761
,2019-08-15 15:51:10,0.065 sec,2,.13E4,77,913.1494122,nan,913.3307772,0.7189256
,2019-08-15 15:51:10,0.139 sec,3,.81E3,77,912.5966543,nan,912.8882653,0.7188445
,2019-08-15 15:51:10,0.208 sec,4,.5E3,78,911.7084617,nan,912.1769190,0.7187138
,2019-08-15 15:51:10,0.275 sec,5,.31E3,79,910.2832189,nan,911.0345970,0.7185036
,2019-08-15 15:51:11,0.340 sec,6,.19E3,79,908.0015234,nan,909.2033969,0.7181662
,2019-08-15 15:51:11,0.412 sec,7,.12E3,79,904.3597864,nan,906.2767038,0.7176241
,2019-08-15 15:51:11,0.474 sec,8,.75E2,79,898.5796696,nan,901.6173791,0.7167559
,2019-08-15 15:51:11,0.535 sec,9,.46E2,79,889.4827221,nan,0.0,0.0


In [37]:
# make predictions using leader model
pred = aml.leader.predict(test)
pred.head()

glm prediction progress: |████████████████████████████████████████████████| 100%


predict
19.5617
19.5616
19.5615
19.5623
19.5692
19.5835
19.6231
19.6919
19.7732
19.8335


In [38]:
pred.shape

(166383, 1)

In [41]:
num_buildings = len(val_raw['building_name'].unique())
print(num_buildings)

19


In [43]:
aml_pred_npy = pred.as_data_frame().as_matrix().reshape((num_buildings, pred.shape[0]//num_buildings))


C:\Users\dmmma\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [45]:
aml_pred_npy.shape

(19, 8757)